In [2]:

from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder \
    .appName("first_app") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()


In [3]:
# Creating the dataframe and subscribing it to the health events topic. We will likely have to create multiple dataframes for the 6 topics


df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "44.201.154.178:9092") \
  .option("subscribe", "health_events") \
  .option("startingOffsets", "latest") \
  .load()



In [ ]:

# A lot of the regular methods you would use on a spark dataframe, because we're using streaming data.
#That's why I'm trying to capture a screenshot of the data and save it to a file. 
#if you can do it in a different way, power to you.

from pyspark.sql.functions import *
from pyspark.sql.types import *
import time 


query = df.selectExpr("CAST(value as STRING)", "topic", "partition", "timestamp")\
    .writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", "file:///") \
    .option("checkpointLocation", "file:///") \
    .start()
timeout_seconds = 20
start_time = time.time()
while (time.time() - start_time) < timeout_seconds and query.isActive:
    time.sleep(1)

if query.isActive == True:
    query.stop()
# # Wait for the query to finish
# query.awaitTermination()
